In [1]:
import os
import pandas as pd


In [21]:
import os
import pandas as pd

# Use current working directory as base
base_dir = os.getcwd()
folder_path = os.path.join(base_dir, "LOG_RETURNS_KALMAN_INTERPOLATED", "monthly")

log_returns = {}

for file in os.listdir(folder_path):
    if file.endswith(".xlsx"):
        file_path = os.path.join(folder_path, file)
        card_name = file.replace(".xlsx", "")
        log_returns[card_name] = pd.read_excel(file_path)

print(f"Loaded {len(log_returns)} log return datasets.")


Loaded 9 log return datasets.


In [22]:
log_returns.keys()

dict_keys(['monthly_data_aggregated_Charizard-Holo_#11_grade_9', 'monthly_data_aggregated_Charizard-Holo_#4_grade_8_FINAL', 'monthly_data_aggregated_Charizard-Holo_#4_grade_9_FINAL', 'monthly_data_aggregated_Charizard-Reverse_Foil_#11_grade_9', 'monthly_data_aggregated_Full_ArtJessie_&_James_#68_grade_10', 'monthly_data_aggregated_Full_ArtM_Charizard_EX_#101_grade_10', 'monthly_data_aggregated_Red_s_Pikachu_#270_(POKEMON_CENTER_20TH_ANNIVERSARY_CAMPAIGN)_grade_10', 'monthly_data_aggregated_Reshiram_&_Charizard_GX_#20_grade_10', 'monthly_data_aggregated_Venusaur-Holo_#15_grade_9_FINAL'])

In [23]:
# Replace with any dataset name to preview
sample_card = list(log_returns.keys())[0]  # First dataset
print(f"Preview of {sample_card}:")
print(log_returns[sample_card].head())


Preview of monthly_data_aggregated_Charizard-Holo_#11_grade_9:
   Date  Log_Returns
0     1    -0.139010
1     2    -0.523014
2     3    -0.363226
3     4     0.165090
4     5     0.078366


In [24]:
!pip install hurst statsmodels numpy pandas


Defaulting to user installation because normal site-packages is not writeable


In [25]:
!pip install fractional

Defaulting to user installation because normal site-packages is not writeable
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for fractional: filename=fractional-0.0.1-py3-none-any.whl size=2514 sha256=0ef7041ef1a40f4f5212cc6bfe6f29fde8297b162caa759188a9a786ba1a726b
  Stored in directory: c:\users\janhr\appdata\local\pip\cache\wheels\19\e8\7a\3ed1246e28624eef16a442cc37483946a8e8e433530a443e7f
Successfully built fractional


In [26]:
!pip install fdiff

Defaulting to user installation because normal site-packages is not writeable


# !! USE THE FOLLOWING TWO HURST EXPONENTS (the most refined so far):

In [12]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.signal import periodogram



def gph_estimator(series, m=None):
    series = series.dropna().values
    n = len(series)

    if m is None:
        m = int(n ** 0.6)  # Slightly higher exponent for robustness

    freqs, spectrum = periodogram(series, scaling='spectrum')

    valid_indices = freqs > 0
    freqs = freqs[valid_indices]
    spectrum = spectrum[valid_indices]

    eps = 1e-10  # Small constant for stability
    log_freqs = np.log(freqs[:m])
    log_spectrum = np.log(spectrum[:m] + eps)

    X = sm.add_constant(-2 * log_freqs)
    model = sm.OLS(log_spectrum, X).fit()

    d = model.params[1] / 2
    hurst_exp = d + 0.5

    return hurst_exp


  # Apply functions to each dataset
hurst_results_gph = {
    card: {
        "Hurst_GPH": gph_estimator(df["Log_Returns"]),

    }
    for card, df in log_returns.items()
}

# Convert to DataFrame for better visualization
hurst_df_gph = pd.DataFrame(hurst_results_gph).T
hurst_df_gph


,Hurst_GPH
monthly_data_aggregated_Full_ArtJessie_&_James_#68_grade_10,0.535019
monthly_data_aggregated_Full_ArtM_Charizard_EX_#101_grade_10,0.650737
monthly_data_aggregated_Charizard-Holo_#11_grade_9,0.725738
monthly_data_aggregated_Reshiram_&_Charizard_GX_#20_grade_10,0.424688
monthly_data_aggregated_Venusaur-Holo_#15_grade_9_FINAL,0.711170
monthly_data_aggregated_Charizard-Holo_#4_grade_9_FINAL,0.615865
monthly_data_aggregated_Charizard-Holo_#4_grade_8_FINAL,0.564224
monthly_data_aggregated_Charizard-Reverse_Foil_#11_grade_9,0.646469
monthly_data_aggregated_Red's_Pikachu_#270_(POKEMON_CENTER_20TH_ANNIVERSARY_CAMPAIGN)_grade_10,0.604148


In [13]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

def local_whittle_estimator(series, m=None):
    series = series.dropna().values  # Remove NaN values
    n = len(series)

    if m is None:
        m = int(n ** 0.8)  # Default bandwidth choice

    # Compute the Fourier frequencies
    freqs = (2 * np.pi * np.arange(1, m + 1)) / n  # Exclude zero frequency

    # Compute the periodogram
    fft_vals = np.fft.fft(series - np.mean(series))  # Demeaned FFT
    periodogram_vals = (np.abs(fft_vals[1:m + 1]) ** 2) / (n / 2)  # Correct normalization

    # Prevent log(0) issues
    eps = 1e-10
    log_freqs = np.log(freqs)
    log_periodogram = np.log(periodogram_vals + eps)  # Avoid log(0) issues

    # OLS regression to estimate d
    X = sm.add_constant(-2 * log_freqs)  # -2 * log(freq) as explanatory variable
    model = sm.OLS(log_periodogram, X).fit()

    d_estimate = model.params[1] / 2  # Estimate of fractional differencing parameter

    hurst_exp = d_estimate + 0.5  # Convert d to Hurst exponent

    return hurst_exp

# Apply function to each Pokémon card dataset
hurst_results_whittle = {
    card: {
        "Hurst_LW": local_whittle_estimator(df["Log_Returns"])  # Apply Local Whittle
    }
    for card, df in log_returns.items()
}

# Convert to DataFrame for better visualization
hurst_df_whittle = pd.DataFrame(hurst_results_whittle).T  # Transpose for readability

hurst_df_whittle


,Hurst_LW
monthly_data_aggregated_Full_ArtJessie_&_James_#68_grade_10,0.232604
monthly_data_aggregated_Full_ArtM_Charizard_EX_#101_grade_10,0.471069
monthly_data_aggregated_Charizard-Holo_#11_grade_9,0.391737
monthly_data_aggregated_Reshiram_&_Charizard_GX_#20_grade_10,0.335857
monthly_data_aggregated_Venusaur-Holo_#15_grade_9_FINAL,0.486646
monthly_data_aggregated_Charizard-Holo_#4_grade_9_FINAL,0.424427
monthly_data_aggregated_Charizard-Holo_#4_grade_8_FINAL,0.473980
monthly_data_aggregated_Charizard-Reverse_Foil_#11_grade_9,0.457306
monthly_data_aggregated_Red's_Pikachu_#270_(POKEMON_CENTER_20TH_ANNIVERSARY_CAMPAIGN)_grade_10,0.562663


# NOW fractal dimension:

# !! USE THE FOLLOWING FRACTAL DIMENSIONS (the most refined):

In [14]:
import numpy as np
import pandas as pd
from scipy.stats import linregress

# **Hall–Wood Fractal Dimension**
def hall_wood_fd(series):
    series = series.dropna().values  # Remove NaN values
    N = len(series)

    if N < 20:  # Skip very short series
        return np.nan

    scales = np.arange(2, min(20, N // 3))  # Use more scales for stability
    log_scales = np.log(scales)
    log_vars = []

    eps = 1e-10  # Small constant to prevent log(0)

    for scale in scales:
        chunks = [series[i:i + scale] for i in range(0, N - scale + 1, scale)]
        chunk_vars = [np.var(chunk) for chunk in chunks if len(chunk) == scale]

        if len(chunk_vars) > 0:
            log_vars.append(np.log(np.mean(chunk_vars) + eps))

    if len(log_vars) < 2:
        return np.nan  # Not enough data points for regression

    log_scales = log_scales[:len(log_vars)]  # Ensure matching lengths
    slope, _, _, _, _ = linregress(log_scales, log_vars)

    return 2 - slope  # Fractal Dimension D

# **Genton Fractal Dimension**
def genton_fd(series):
    series = series.dropna().values  # Remove NaN values
    N = len(series)

    if N < 20:
        return np.nan

    scales = np.arange(2, min(20, N // 3))  # More scales
    log_scales = np.log(scales)
    log_range = []

    eps = 1e-10  # Small constant for stability

    for scale in scales:
        chunks = [series[i:i + scale] for i in range(0, N - scale + 1, scale)]
        chunk_ranges = [np.percentile(chunk, 75) - np.percentile(chunk, 25) for chunk in chunks if len(chunk) == scale]  # Use IQR

        if len(chunk_ranges) > 0:
            log_range.append(np.log(np.mean(chunk_ranges) + eps))

    if len(log_range) < 2:
        return np.nan  # Not enough data points for regression

    log_scales = log_scales[:len(log_range)]  # Ensure matching lengths
    slope, _, _, _, _ = linregress(log_scales, log_range)

    return 2 - slope  # Fractal Dimension D

# **Apply both estimators to all datasets**
fractal_results = {
    card: {
        "Hall_Wood_FD": hall_wood_fd(df["Log_Returns"]),
        "Genton_FD": genton_fd(df["Log_Returns"])
    }
    for card, df in log_returns.items()
}

# Convert to DataFrame for visualization
fractal_df = pd.DataFrame(fractal_results).T
fractal_df


,Hall_Wood_FD,Genton_FD
monthly_data_aggregated_Full_ArtJessie_&_James_#68_grade_10,1.791974,1.943319
monthly_data_aggregated_Full_ArtM_Charizard_EX_#101_grade_10,1.758632,1.817517
monthly_data_aggregated_Charizard-Holo_#11_grade_9,1.857317,1.912792
monthly_data_aggregated_Reshiram_&_Charizard_GX_#20_grade_10,1.863206,1.805315
monthly_data_aggregated_Venusaur-Holo_#15_grade_9_FINAL,1.878121,1.866516
monthly_data_aggregated_Charizard-Holo_#4_grade_9_FINAL,1.845175,1.893031
monthly_data_aggregated_Charizard-Holo_#4_grade_8_FINAL,1.891009,1.872086
monthly_data_aggregated_Charizard-Reverse_Foil_#11_grade_9,1.941555,1.815142
monthly_data_aggregated_Red's_Pikachu_#270_(POKEMON_CENTER_20TH_ANNIVERSARY_CAMPAIGN)_grade_10,1.670549,1.813097


# !! USE THE FOLLOWING ONLY FOR THE APPROX. ENTROPY:

In [15]:
import numpy as np
import pandas as pd

# Function to compute Approximate Entropy (ApEn)
def approximate_entropy(series, m=2, r=None):
    series = series.dropna().values  # Remove NaNs
    N = len(series)

    eps = 1e-10  # Small constant for numerical stability

    if N < m + 1:
        return np.nan  # Not enough data

    if r is None:
        r = 0.2 * np.std(series)  # Set default tolerance

    # Create patterns
    patterns_m = np.array([series[i:i + m] for i in range(N - m + 1)])
    patterns_m1 = np.array([series[i:i + m + 1] for i in range(N - m)])

    def phi(patterns):
        distances = np.abs(patterns[:, None, :] - patterns[None, :, :]).max(axis=2)
        C = np.sum(distances <= r, axis=1) / (N - len(patterns) + 1)
        return np.mean(np.log(C + eps))  # Prevent log(0) errors

    return abs(phi(patterns_m) - phi(patterns_m1))  # ApEn formula

# **Apply to all datasets**
approx_entropy_results = {
    card: approximate_entropy(df["Log_Returns"])
    for card, df in log_returns.items()
}

# Convert to DataFrame for better visualization
approx_entropy_df = pd.DataFrame.from_dict(approx_entropy_results, orient="index", columns=["Approx_Entropy"])

approx_entropy_df


,Approx_Entropy
monthly_data_aggregated_Full_ArtJessie_&_James_#68_grade_10,1.278124
monthly_data_aggregated_Full_ArtM_Charizard_EX_#101_grade_10,1.132805
monthly_data_aggregated_Charizard-Holo_#11_grade_9,1.146083
monthly_data_aggregated_Reshiram_&_Charizard_GX_#20_grade_10,0.861378
monthly_data_aggregated_Venusaur-Holo_#15_grade_9_FINAL,1.116615
monthly_data_aggregated_Charizard-Holo_#4_grade_9_FINAL,1.051609
monthly_data_aggregated_Charizard-Holo_#4_grade_8_FINAL,1.172723
monthly_data_aggregated_Charizard-Reverse_Foil_#11_grade_9,1.079883
monthly_data_aggregated_Red's_Pikachu_#270_(POKEMON_CENTER_20TH_ANNIVERSARY_CAMPAIGN)_grade_10,1.026561


# Now finally the efficiency index:

# !! USE THE FOLLOWING FOR EI - THE BEST CURATED:

In [ ]:
import numpy as np
import pandas as pd

# Expected values for an efficient market
M_star = {
    "Hurst": 0.5,
    "Fractal_Dimension": 1.5,
    "Approx_Entropy": 1.0
}

# Ranges for normalization
R_values = {
    "Hurst": 1.0,  # Hurst exponent range [0,1]
    "Fractal_Dimension": 1.0,  # Fractal dimension range [1,2]
    "Approx_Entropy": 2.0  # Entropy range as per Kristoufek & Vosvrda
}

# Load efficiency measures into a DataFrame (Assuming you have these DataFrames ready)
efficiency_data = pd.DataFrame({
    "Hurst_GPH": hurst_df_gph["Hurst_GPH"],  # GPH estimator
    "Hurst_LW": hurst_df_whittle["Hurst_LW"],  # Local Whittle estimator
    "Hall_Wood_FD": fractal_df["Hall_Wood_FD"],
    "Genton_FD": fractal_df["Genton_FD"],
    "Approx_Entropy": approx_entropy_df["Approx_Entropy"]
})

# Compute Efficiency Index (EI) function
def compute_efficiency_index(row):
    EI_values = []

    # Compute separately for both Hurst estimators (GPH & Local Whittle)
    for hurst in ["Hurst_GPH", "Hurst_LW"]:
        EI_values.append(((row[hurst] - M_star["Hurst"]) / R_values["Hurst"])**2)

    # Compute separately for both Fractal Dimension estimators
    for fd in ["Hall_Wood_FD", "Genton_FD"]:
        EI_values.append(((row[fd] - M_star["Fractal_Dimension"]) / R_values["Fractal_Dimension"])**2)

    # Compute for Approximate Entropy
    EI_values.append(((row["Approx_Entropy"] - M_star["Approx_Entropy"]) / R_values["Approx_Entropy"])**2)

    # Final Efficiency Index calculation
    return np.sqrt(sum(EI_values))

# Apply function to all rows
efficiency_data["Efficiency_Index"] = efficiency_data.apply(compute_efficiency_index, axis=1)

efficiency_data

,Hurst_GPH,Hurst_LW,Hall_Wood_FD,Genton_FD,Approx_Entropy,Efficiency_Index
monthly_data_aggregated_Full_ArtJessie_&_James_#68_grade_10,0.535019,0.232604,1.791974,1.943319,1.278124,0.611430
monthly_data_aggregated_Venusaur-Holo_#15_grade_9_FINAL,0.711170,0.486646,1.878121,1.866516,1.116615,0.570509
monthly_data_aggregated_Charizard-Holo_#11_grade_9,0.725738,0.391737,1.857317,1.912792,1.146083,0.605051
monthly_data_aggregated_Charizard-Holo_#4_grade_8_FINAL,0.564224,0.473980,1.891009,1.872086,1.172723,0.550996
monthly_data_aggregated_Charizard-Reverse_Foil_#11_grade_9,0.646469,0.457306,1.941555,1.815142,1.079883,0.564940
monthly_data_aggregated_Charizard-Holo_#4_grade_9_FINAL,0.615865,0.424427,1.845175,1.893031,1.051609,0.541684
monthly_data_aggregated_Reshiram_&_Charizard_GX_#20_grade_10,0.424688,0.335857,1.863206,1.805315,0.861378,0.512401
monthly_data_aggregated_Red's_Pikachu_#270_(POKEMON_CENTER_20TH_ANNIVERSARY_CAMPAIGN)_grade_10,0.604148,0.562663,1.670549,1.813097,1.026561,0.376917
monthly_data_aggregated_Full_ArtM_Charizard_EX_#101_grade_10,0.650737,0.471069,1.758632,1.817517,1.132805,0.442352


## top functioning bootstrap (just wait 30 mins:):

In [ ]:
import numpy as np
import pandas as pd

# Function to compute the Efficiency Index
def compute_efficiency_index(row, M_star, R_values):
    EI_values = []

    # Compute for both Hurst exponents
    for hurst in ["Hurst_GPH", "Hurst_LW"]:
        EI_values.append(((row[hurst] - M_star["Hurst"]) / R_values["Hurst"])**2)

    # Compute for both Fractal Dimensions
    for fd in ["Hall_Wood_FD", "Genton_FD"]:
        EI_values.append(((row[fd] - M_star["Fractal_Dimension"]) / R_values["Fractal_Dimension"])**2)

    # Compute for Approximate Entropy
    EI_values.append(((row["Approx_Entropy"] - M_star["Approx_Entropy"]) / R_values["Approx_Entropy"])**2)

    return np.sqrt(sum(EI_values))

# Define expected values and ranges for efficiency measures
M_star = {"Hurst": 0.5, "Fractal_Dimension": 1.5, "Approx_Entropy": 1}
R_values = {"Hurst": 1, "Fractal_Dimension": 1, "Approx_Entropy": 2}  # Entropy range is double

# Step 1: Compute original Efficiency Index
efficiency_data["Efficiency_Index"] = efficiency_data.apply(compute_efficiency_index, axis=1, M_star=M_star, R_values=R_values)

# Function to perform bootstrapping for a given card
def bootstrap_efficiency(log_returns, num_bootstraps=1000):
    n = len(log_returns)
    bootstrap_EI = []

    for _ in range(num_bootstraps):
        # Step 2: Generate a bootstrapped sample with replacement
        bootstrapped_series = np.random.choice(log_returns, size=n, replace=True)

        # Step 3: Compute Efficiency Index for bootstrapped series
        bootstrapped_measures = {
            "Hurst_GPH": gph_estimator(pd.Series(bootstrapped_series)),
            "Hurst_LW": local_whittle_estimator(pd.Series(bootstrapped_series)),
            "Hall_Wood_FD": hall_wood_fd(pd.Series(bootstrapped_series)),
            "Genton_FD": genton_fd(pd.Series(bootstrapped_series)),
            "Approx_Entropy": approximate_entropy(pd.Series(bootstrapped_series))
        }
        bootstrapped_EI = compute_efficiency_index(bootstrapped_measures, M_star, R_values)
        bootstrap_EI.append(bootstrapped_EI)

    return bootstrap_EI

# Apply bootstrapping to all cards
bootstrap_results = {}
for card, df in log_returns.items():
    bootstrap_EI_values = bootstrap_efficiency(df["Log_Returns"])

    # Step 5: Compute statistical significance
    q05, q95 = np.percentile(bootstrap_EI_values, [5, 95])
    original_EI = efficiency_data.loc[card, "Efficiency_Index"]

    # Compute p-value based on rank of original EI
    r = sum(bootstrap_EI_values < original_EI) + 1  # Rank of original EI
    p_value = 1 - 2 * abs(r / (len(bootstrap_EI_values) + 1) - 0.5)

    # Store results
    bootstrap_results[card] = {
        "EI": original_EI,
        "EI_Boot_Mean": np.mean(bootstrap_EI_values),
        "EI_Boot_SD": np.std(bootstrap_EI_values),
        "q05": q05,
        "q95": q95,
        "p_value": p_value
    }

# Convert results to DataFrame
bootstrap_results_df = pd.DataFrame.from_dict(bootstrap_results, orient="index")
bootstrap_results_df

,EI,EI_Boot_Mean,EI_Boot_SD,q05,q95,p_value
monthly_data_aggregated_Full_ArtJessie_&_James_#68_grade_10,0.611430,0.581213,0.149618,0.351218,0.839313,0.823177
monthly_data_aggregated_Venusaur-Holo_#15_grade_9_FINAL,0.570509,0.475996,0.072252,0.357950,0.597328,0.183816
monthly_data_aggregated_Charizard-Holo_#11_grade_9,0.605051,0.497250,0.070422,0.381081,0.611658,0.107892
monthly_data_aggregated_Charizard-Holo_#4_grade_8_FINAL,0.550996,0.492415,0.073295,0.377984,0.616652,0.393606
monthly_data_aggregated_Charizard-Reverse_Foil_#11_grade_9,0.564940,0.415412,0.070533,0.300538,0.528401,0.031968
monthly_data_aggregated_Charizard-Holo_#4_grade_9_FINAL,0.541684,0.483810,0.072406,0.375056,0.608083,0.425574
monthly_data_aggregated_Reshiram_&_Charizard_GX_#20_grade_10,0.512401,0.472116,0.098560,0.320475,0.635251,0.663337
monthly_data_aggregated_Red's_Pikachu_#270_(POKEMON_CENTER_20TH_ANNIVERSARY_CAMPAIGN)_grade_10,0.376917,0.497013,0.102679,0.341121,0.667107,0.237762
monthly_data_aggregated_Full_ArtM_Charizard_EX_#101_grade_10,0.442352,0.471018,0.073163,0.360213,0.596045,0.701299


In [ ]:
# Compute the alternative inefficiency measure (distance in SDs)
bootstrap_results_df["↑distance↑"] = (
    abs(bootstrap_results_df["EI"] - bootstrap_results_df["EI_Boot_Mean"]) /
    bootstrap_results_df["EI_Boot_SD"]
)

# Sort by highest inefficiency (most inefficient cards first)
bootstrap_results_df = bootstrap_results_df.sort_values(by="↑distance↑", ascending=True)


bootstrap_results_df

,EI,EI_Boot_Mean,EI_Boot_SD,q05,q95,p_value,↑distance↑
monthly_data_aggregated_Full_ArtJessie_&_James_#68_grade_10,0.611430,0.581213,0.149618,0.351218,0.839313,0.823177,0.201959
monthly_data_aggregated_Full_ArtM_Charizard_EX_#101_grade_10,0.442352,0.471018,0.073163,0.360213,0.596045,0.701299,0.391810
monthly_data_aggregated_Reshiram_&_Charizard_GX_#20_grade_10,0.512401,0.472116,0.098560,0.320475,0.635251,0.663337,0.408734
monthly_data_aggregated_Charizard-Holo_#4_grade_8_FINAL,0.550996,0.492415,0.073295,0.377984,0.616652,0.393606,0.799245
monthly_data_aggregated_Charizard-Holo_#4_grade_9_FINAL,0.541684,0.483810,0.072406,0.375056,0.608083,0.425574,0.799285
monthly_data_aggregated_Red's_Pikachu_#270_(POKEMON_CENTER_20TH_ANNIVERSARY_CAMPAIGN)_grade_10,0.376917,0.497013,0.102679,0.341121,0.667107,0.237762,1.169635
monthly_data_aggregated_Venusaur-Holo_#15_grade_9_FINAL,0.570509,0.475996,0.072252,0.357950,0.597328,0.183816,1.308107
monthly_data_aggregated_Charizard-Holo_#11_grade_9,0.605051,0.497250,0.070422,0.381081,0.611658,0.107892,1.530784
monthly_data_aggregated_Charizard-Reverse_Foil_#11_grade_9,0.564940,0.415412,0.070533,0.300538,0.528401,0.031968,2.119983


 Next Steps for Further Analysis

Compare Inefficient Cards to Trading Volume

    Do the most inefficient cards have the lowest liquidity?
    Does high inefficiency correlate with low sales activity?

Visualize Price Trajectories of Inefficient Cards

    Do the most inefficient cards have erratic price spikes & drops?
    If so, that confirms inefficiency.

Check If Inefficiency Persists Over Time

    Run the same bootstrapping analysis on monthly data.
    If inefficiency disappears, markets may be inefficient in the short term but efficient in the long run.